# Feature extraction from augmented data

In [1]:
import numpy as np
import mne
from scipy import signal
from scipy.interpolate import RectBivariateSpline
from mne.filter import resample, filter_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lspopt import spectrogram_lspopt
from matplotlib.colors import Normalize, ListedColormap

import logging
LOGGING_TYPES = dict(DEBUG=logging.DEBUG, INFO=logging.INFO, WARNING=logging.WARNING,
                     ERROR=logging.ERROR, CRITICAL=logging.CRITICAL)
logger = logging.getLogger('yasa')

# %matplotlib qt


In [2]:
# load reference_df     
reference_df = pd.read_csv("reference_df.csv", index_col="name")
reference_df.head(3)

,hypno,df_feat,eeg
name,,,
P18_N3 L,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P18_N3 L.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...
P18_N2 R,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P18_N2 R.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...
P17_N2 L,/Users/amirhosseindaraie/Desktop/data/synced-h...,feature/P17_N2 L.csv,/Users/amirhosseindaraie/Desktop/data/autoscor...


In [3]:
# load csv
rankings_df = pd.read_csv("rankings_df.csv", index_col="method_name")
rankings_df.head(3)


,sb,ab,bs,ag,sg,gs,ba,ta_b,alpha,ga,...,mean_psd,kurt,E,WEn,renyi,mean_distance,diffEnt,skew,tsallisEnt,mean
method_name,,,,,,,,,,,,,,,,,,,,,
f_classif,1.0,2.0,3.0,4.0,5.0,10.0,11.0,6.0,7.0,22.0,...,66.0,72.0,67.0,69.0,70.0,65.0,71.0,73.0,74.0,75.0
MI,1.0,2.0,3.0,4.0,8.0,9.0,5.0,7.0,17.0,6.0,...,70.0,67.0,73.0,68.0,71.0,74.0,72.0,66.0,69.0,75.0
chiSqr,2.0,1.0,3.0,6.0,9.0,4.0,8.0,16.0,5.0,7.0,...,66.0,64.0,67.0,71.0,68.0,72.0,69.0,73.0,74.0,75.0


In [5]:
def sliding_window(data, sf, window, step=None, axis=-1):
    """Calculate a sliding window of a 1D or 2D EEG signal.
    .. versionadded:: 0.1.7
    Parameters
    ----------
    data : numpy array
        The 1D or 2D EEG data.
    sf : float
        The sampling frequency of ``data``.
    window : int
        The sliding window length, in seconds.
    step : int
        The sliding window step length, in seconds.
        If None (default), ``step`` is set to ``window``,
        which results in no overlap between the sliding windows.
    axis : int
        The axis to slide over. Defaults to the last axis.
    Returns
    -------
    times : numpy array
        Time vector, in seconds, corresponding to the START of each sliding
        epoch in ``strided``.
    strided : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window, shape (n_epochs, ..., n_samples).
    Notes
    -----
    This is a wrapper around the
    :py:func:`numpy.lib.stride_tricks.as_strided` function.
    Examples
    --------
    With a 1-D array
    >>> import numpy as np
    >>> from yasa import sliding_window
    >>> data = np.arange(20)
    >>> times, epochs = sliding_window(data, sf=1, window=5)
    >>> times
    array([ 0.,  5., 10., 15.])
    >>> epochs
    array([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
    >>> sliding_window(data, sf=1, window=5, step=1)[1]
    array([[ 0,  1,  2,  3,  4],
        [ 2,  3,  4,  5,  6],
        [ 4,  5,  6,  7,  8],
        [ 6,  7,  8,  9, 10],
        [ 8,  9, 10, 11, 12],
        [10, 11, 12, 13, 14],
        [12, 13, 14, 15, 16],
        [14, 15, 16, 17, 18]])
    >>> sliding_window(data, sf=1, window=11)[1]
    array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
    With a N-D array
    >>> np.random.seed(42)
    >>> # 4 channels x 20 samples
    >>> data = np.random.randint(-100, 100, size=(4, 20))
    >>> epochs = sliding_window(data, sf=1, window=10)[1]
    >>> epochs.shape  # shape (n_epochs, n_channels, n_samples)
    (2, 4, 10)
    >>> epochs
    array([[[  2,  79,  -8, -86,   6, -29,  88, -80,   2,  21],
            [-13,  57, -63,  29,  91,  87, -80,  60, -43, -79],
            [-50,   7, -46, -37,  30, -50,  34, -80, -28,  66],
            [ -9,  10,  87,  98,  71, -93,  74, -66, -20,  63]],
        [[-26, -13,  16,  -1,   3,  51,  30,  49, -48, -99],
            [-12, -52, -42,  69,  87, -86,  89,  89,  74,  89],
            [-83,  31, -12, -41, -87, -92, -11, -48,  29, -17],
            [-51,   3,  31, -99,  33, -47,   5, -97, -47,  90]]])
    """
    from numpy.lib.stride_tricks import as_strided

    assert axis <= data.ndim, "Axis value out of range."
    assert isinstance(sf, (int, float)), "sf must be int or float"
    assert isinstance(window, (int, float)), "window must be int or float"
    assert isinstance(step, (int, float, type(None))), (
        "step must be int, " "float or None."
    )
    if isinstance(sf, float):
        assert sf.is_integer(), "sf must be a whole number."
        sf = int(sf)
    assert isinstance(axis, int), "axis must be int."

    # window and step in samples instead of points
    window *= sf
    step = window if step is None else step * sf

    if isinstance(window, float):
        assert window.is_integer(), "window * sf must be a whole number."
        window = int(window)

    if isinstance(step, float):
        assert step.is_integer(), "step * sf must be a whole number."
        step = int(step)

    assert step >= 1, "Stepsize may not be zero or negative."
    assert window < data.shape[axis], (
        "Sliding window size may not exceed " "size of selected axis"
    )

    # Define output shape
    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / step - window / step + 1).astype(int)
    shape.append(window)

    # Calculate strides and time vector
    strides = list(data.strides)
    strides[axis] *= step
    strides.append(data.strides[axis])
    strided = as_strided(data, shape=shape, strides=strides)
    t = np.arange(strided.shape[-2]) * (step / sf)

    # Swap axis: n_epochs, ..., n_samples
    if strided.ndim > 2:
        strided = np.rollaxis(strided, -2, 0)
    return t, strided


# Augmentation for all recordings

In [19]:
def augment_data(data_win, hypno_30s, class_to_augment=0, name="Subject [ ]"):
    """
    This function augments the epochs with class `class_to_augment` (int) in `data_win` (matrix) until the count of `class_to_augment` reaches the count of dominate class in that sleep session. It returns the new augmented matrix `data_win_aug` and the new hypnogram `hypno_30s_aug`.

    Parameters
    ----------
    class_to_augment : int
        The class number that we want to augment
    data_win : numpy array (matrix)
        The `data_win` matrix that contains n epochs (rows) and each epoch 30 seconds of data (columns). The size is `(epochs x datapoints)` usually `(800,7680)`.
    hypno_30s : numpy array (vector)
        The hypnogram associated with `data_win`. This is usually in the size if `(epochs,)`.
    name: str, optional
        Subject name

    Returns
    -------
    data_win_aug:
        Augmented data_win that has size of `(augmented_epochs x datapoints)`.
    hypno_30s_aug
        Augmented hypno_30s that is in the size of `(augmented_epochs,)`.
    """
    data_win_aug = np.array([])
    hypno_30s_aug = np.array([])
    method_used = 0
    _, counts = np.unique(hypno_30s, return_counts=True)
    dominate_class_count = np.max(counts)
    current_class_count = data_win[hypno_30s == class_to_augment].shape[0]

    if current_class_count < dominate_class_count * 0.9:
        while current_class_count < dominate_class_count * 0.9:
            if method_used == 0:
                # here we inverse the signal from end to start : 1 -> 1'
                augmented_win = data_win[hypno_30s == class_to_augment][:, ::-1]
                data_win_aug = np.vstack([data_win, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 1:
                #  here we divide signal into 2 : 1,2 -> 2,1
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 2)]
                part2_win = data_win[hypno_30s == class_to_augment][:, int(7680 / 2) :]
                augmented_win = np.hstack([part2_win, part1_win])
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 2:
                #  here we divide signal into 2 : 1,2 -> 2',1'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 2)]
                part2_win = data_win[hypno_30s == class_to_augment][:, int(7680 / 2) :]
                augmented_win = np.hstack([part2_win[:, ::-1], part1_win[:, ::-1]])
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 3:
                #  here we divide signal into 3 : 1,2,3 -> 3,1,2
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part3_win[:, :], part1_win[:, :], part2_win[:, :]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 4:
                #  here we divide signal into 3 : 1,2,3 -> 3',1',2'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part3_win[:, ::-1], part1_win[:, ::-1], part2_win[:, ::-1]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 5:
                # here we divide signal into 3 : 1,2,3 -> 2,3,1
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part2_win[:, :], part3_win[:, :], part1_win[:, :]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 6:
                # here we divide signal into 3 : 1,2,3 -> 2',3',1'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part2_win[:, ::-1], part3_win[:, ::-1], part1_win[:, ::-1]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 7:
                # here we divide signal into 3 : 1,2,3 -> 3,2,1
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part3_win[:, :], part2_win[:, :], part1_win[:, :]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 8:
                # here we divide signal into 3 : 1,2,3 -> 3',2',1'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part3_win[:, ::-1], part2_win[:, ::-1], part1_win[:, ::-1]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 9:
                # here we divide signal into 3 : 1,2,3 -> 1',3',2'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part1_win[:, ::-1], part3_win[:, ::-1], part2_win[:, ::-1]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 10:
                # here we divide signal into 3 : 1,2,3 -> 1',2',3'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 3)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3) : int(7680 / 3 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 3 * 2) :
                ]
                augmented_win = np.hstack(
                    [part1_win[:, ::-1], part2_win[:, ::-1], part3_win[:, ::-1]]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 11:
                # here we divide signal into 4 : 1,2,3,4 -> 4,3,2,1
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 4)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4) : int(7680 / 4 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4 * 2) : int(7680 / 4 * 3)
                ]
                part4_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4 * 3) :
                ]
                augmented_win = np.hstack(
                    [
                        part4_win[:, ::-1],
                        part3_win[:, :],
                        part2_win[:, :],
                        part1_win[:, :],
                    ]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            elif method_used == 12:
                # here we divide signal into 4 : 1,2,3,4 -> 4',3',2',1'
                part1_win = data_win[hypno_30s == class_to_augment][:, : int(7680 / 4)]
                part2_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4) : int(7680 / 4 * 2)
                ]
                part3_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4 * 2) : int(7680 / 4 * 3)
                ]
                part4_win = data_win[hypno_30s == class_to_augment][
                    :, int(7680 / 4 * 3) :
                ]
                augmented_win = np.hstack(
                    [
                        part4_win[:, ::-1],
                        part3_win[:, ::-1],
                        part2_win[:, ::-1],
                        part1_win[:, ::-1],
                    ]
                )
                data_win_aug = np.vstack([data_win_aug, augmented_win])
                hypno_30s_aug = np.append(
                    hypno_30s_aug, np.ones(augmented_win.shape[0]) * class_to_augment
                )
                method_used += 1

            else:
                break

            current_class_count = data_win_aug[hypno_30s_aug == class_to_augment].shape[
                0
            ]

    else:
        data_win_aug = data_win
        hypno_30s_aug = hypno_30s

    print(f">> Feature augmentation @ {name} Class {class_to_augment} ")
    print(f"curr: {current_class_count}, domin: {dominate_class_count}")
    return data_win_aug, hypno_30s_aug


def augment_data_wrapper(data_win, hypno_30s, name="Subject [ ]"):
    """
    This function is a wrapper for `augment_data` function. The main idea is to repeat `augment_data` function for all classes and output an augmented `data_win`.

    Parameters
    ----------
    data_win : numpy array (matrix)
        The `data_win` matrix that contains n epochs (rows) and each epoch 30 seconds of data (columns). The size is `(epochs x datapoints)` usually `(800,7680)`.
    hypno_30s : numpy array (vector)
        The hypnogram associated with `data_win`. This is usually in the size if `(epochs,)`.
    name: str, optional
        Subject name

    Returns
    -------
    data_win_aug:
        Augmented data_win that has size of `(augmented_epochs x datapoints)`.
    hypno_30s_aug
        Augmented hypno_30s that is in the size of `(augmented_epochs,)`.
    """
    data_win_aug, hypno_30s_aug = augment_data(
        class_to_augment=0, data_win=data_win, hypno_30s=hypno_30s, name=name
    )
    data_win_aug, hypno_30s_aug = augment_data(
        class_to_augment=1, data_win=data_win_aug, hypno_30s=hypno_30s_aug, name=name
    )
    data_win_aug, hypno_30s_aug = augment_data(
        class_to_augment=2, data_win=data_win_aug, hypno_30s=hypno_30s_aug, name=name
    )
    data_win_aug, hypno_30s_aug = augment_data(
        class_to_augment=3, data_win=data_win_aug, hypno_30s=hypno_30s_aug, name=name
    )
    data_win_aug, hypno_30s_aug = augment_data(
        class_to_augment=4, data_win=data_win_aug, hypno_30s=hypno_30s_aug, name=name
    )
    return data_win_aug, hypno_30s_aug


In [40]:
for i in range(0, len(reference_df)):
    # to load information of each night:
    name = reference_df.iloc[i].name
    hypno_30s_loc = reference_df.iloc[i].hypno
    df_feat_loc = reference_df.iloc[i].df_feat
    eeg_loc = reference_df.iloc[i].eeg

    # to load data of each night:
    hypno_30s = np.loadtxt(hypno_30s_loc)[:, 0]
    df_feat = pd.read_csv(df_feat_loc, index_col=False)
    raw = mne.io.read_raw_edf(eeg_loc, preload=True, verbose=0)
    raw.pick_types(eeg=True)
    sf = raw.info["sfreq"]
    raw.filter(0.5, 45)
    data = raw._data * 1e6  # : extract the data and convert from V to uV

    times = np.arange(data.size) / sf  # : time vector in seconds
    times, data_win = sliding_window(
        data[0], sf, window=30
    )  # : convert the EEG data to 30-sec data
    times /= 60  # : convert times to minutes

    # to filter artifact epochs in each night:
    data_win = data_win[hypno_30s != -1, :]
    df_feat = df_feat.loc[hypno_30s != -1]
    hypno_30s = hypno_30s[hypno_30s != -1]

    # to change 5 to 4 in socrings that assigned 5 to REM instead of 4:
    hypno_30s[hypno_30s == 5] = 4

    # to see the class imbalance:
    stages, counts = np.unique(
        hypno_30s, return_counts=True
    )  # to see class balance of each night:

    # printing histogram of classes in this night:
    print(f"Sleep stages for {name} before augmentation:")
    print(stages, counts)

    # Augment data_win:
    data_win_aug, hypno_30s_aug = augment_data_wrapper(data_win, hypno_30s, name=name)

    # Save augmented data to same location as data (EEG & hypno). Only add " aug" to their end.
    np.savetxt(eeg_loc.split(".")[0] + " aug.txt", data_win_aug, delimiter=",")

    textfile = open(hypno_30s_loc.split(".")[0] + " aug.txt", "w")
    for element in hypno_30s_aug:
        textfile.write(f"{int(element)}" + "\n")
    textfile.close()

    print(eeg_loc.split(".")[0] + " aug.csv")
    print(hypno_30s_loc.split(".")[0] + " aug.txt")

    fig, ax = plt.subplots(figsize=(15, 13))
    stages, counts = np.unique(hypno_30s_aug, return_counts=True)
    ax.bar(stages, counts, color="blueviolet", label="After")
    stages, counts = np.unique(hypno_30s, return_counts=True)
    ax.bar(stages, counts, color="red", label="Before")
    ax.set(xticks=np.arange(0, 4 + 1, 1), xticklabels=["Wake", "N1", "N2", "N3", "REM"])
    ax.tick_params(
        axis="x", labelsize=13, labelrotation=20, labelcolor="green", width=3
    )
    ax.tick_params(axis="y", labelsize=13, labelrotation=20, labelcolor="darkmagenta")
    plt.xlabel("Sleep stage")
    plt.ylabel("Count")
    plt.title(f"Sleep stages for {name} after augmentation")
    plt.legend()
    plt.tight_layout()
    plt.savefig(
        f"stage_distribution augment/stage_distribution_count {name} after aug.png"
    )
    plt.close()


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Sleep stages for P18_N3 L before augmentation:
[0. 1. 2. 3. 4.] [ 45  66 441 135 262]
>> Feature augmentation @ P18_N3 L Class 0 
curr: 405, domin: 441
>> Feature augmentation @ P18_N3 L Class 1 
curr: 462, domin: 441
>> Feature augmentation @ P18_N3 L Class 2 
curr: 441, domin: 462
>> Feature augmentation @ P18_N3 L Class 3 
curr: 540, domin: 462
>> Feature augmentation @ P18_N3 L Class 4 
curr: 524, domin: 540
/Users/amirhossein

/var/folders/c4/x03my2dj70s46m109bn58l2m0000gn/T/ipykernel_73698/25031925.py:11: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(eeg_loc, preload=True, verbose=0)


Sleep stages for QS 19 L before augmentation:
[0. 1. 2. 3. 4.] [ 19  44 531 105  70]
>> Feature augmentation @ QS 19 L Class 0 
curr: 266, domin: 531
>> Feature augmentation @ QS 19 L Class 1 
curr: 484, domin: 531
>> Feature augmentation @ QS 19 L Class 2 
curr: 531, domin: 531
>> Feature augmentation @ QS 19 L Class 3 
curr: 525, domin: 531
>> Feature augmentation @ QS 19 L Class 4 
curr: 490, domin: 531
/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/19/EEG L aug.csv
/Users/amirhosseindaraie/Desktop/data/synced-hypnos-merged/session_19_synced aug.txt
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Up